# Correlation (Корреляционная зависимость)



I am learning to find correlations.  This work may contain errors, if you find them, please write in the comments.

( Учусь находить кореляции.

Эта работа может содержать ошибки, если вы их найдете, то напишите пожалуйста в коментарии.)

In [ ]:
import numpy as np 
import pandas as pd
import math

$ \overline{x} $ - arithmetic mean (арефметическое среднее)

$r_{xy} = \dfrac{\overline{xy} - \overline{x} \cdot \overline{y}}{\sigma_{x} \cdot \sigma_{y}}$ - coefficient correlation (коэффициент корреляции), где $\sigma_{x}, \sigma_{y}$ - standard deviation (среднеквадратические отклонения)

$\sigma_{x} = \sqrt{D[x]}$ , где $D[x]$ - variance (дисперсия случайной величины)

$D[x] = \dfrac{1}{n} \cdot \sum^{n}_{i=1} (x_{i} - \overline{x})^{2} $

In [ ]:
def coefficient_correlation(col_x, col_y):
    n = len(col_x)
    # find mean x and y (находим среднее значение в обоих колонках)
    x = 1/n * sum(col_x)
    y = 1/n * sum(col_y)
    # find mean xy (находим среднее значение произведения xy)
    xy = 1/n * sum([col_x[i]*col_y[i] for i in range(n)])
    # find variance (находим дисперсию)
    D_x = 1/n * sum([(col_x[i]- x)**2 for i in range(n)])
    D_y = 1/n * sum([(col_y[i]- y)**2 for i in range(n)])
    # find sigma (находим sigma)
    sigma_x = math.sqrt(D_x)
    sigma_y = math.sqrt(D_y)
    return (xy - x * y)/(sigma_x * sigma_y)

In [ ]:
# Example r_xy = 0.9859
col_x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
col_y = [107, 109, 110, 113, 120, 122, 123, 128, 136, 140, 145,150]
coefficient_correlation(col_x, col_y) 

Let's choose the columns we need (Выберем нужные нам колонки) 

In [ ]:
columns = ["danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness",
           "liveness", "valence", "tempo", "duration_ms", "time_signature", "genre"]
data = pd.read_csv("../input/dataset-of-songs-in-spotify/genres_v2.csv", 
                   usecols=columns)
data.head()

Check for null data (Проверим на наличие отсутствующих даннух)

In [ ]:
data.isnull().count() # нет отсутствующих данных 

In [ ]:
# Для удобства, преобразуем колонку "genre" к котегориальному типу данных
genre_unique = list(data.genre.unique())
genre = {} # создаем словарь
i = 0
for name in genre_unique:
    genre[name] = i
    i = i + 1

new_genre = [] # используя ранее созданный словарь, создаем список значений
for row in data.genre:
    new_genre.append(genre[row])

data["new_genre"] = new_genre # добовляем новый столбец, состоящий из списка значений
data = data.drop("genre", axis=1) # удаляем столбец "genre"

print(genre)

In [ ]:
data.head()

In [ ]:
df_genre = pd.DataFrame({"genre": list(data.groupby("new_genre").new_genre.count())}, index=genre_unique)

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(20,5))
df_plot = sns.barplot(x=df_genre.index, y=df_genre["genre"])

# Создаем DataFrame содержащий коэффициенты корреляции

In [ ]:
def col_coefficient_correlation(col_x, data):
    coefficient = []
    for col in list(data.columns):
        coefficient.append(coefficient_correlation(data[col_x], data[col]))
    return coefficient

In [ ]:
index = list(data.columns)
columns = {}
for col in index:
    columns[col] = col_coefficient_correlation(col, data)
    print(col, " - finish")

In [ ]:
df = pd.DataFrame(columns, index=index)
df

In [ ]:
plt.figure(figsize=(14,10))
tab = sns.heatmap(data=df, vmin=-1, vmax=1, annot=True)

Вывод: найдены слабые зависимости: 
1. Прямая зависимость: loudness - energy (0.6); instrumentalness - duration_ms (0.6); energy - new_genre (0.54)
2. Обратная зависимость: energy - acousticness (-0.5)